In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
X = pd.read_csv('neg_A0201.txt', header=None)
X_train, X_test = train_test_split(X, test_size=0.1, random_state=42)